# In this Notebook, I'll write the script for training the Order-Planner Model defined in the base referenced paper
-------------------------------------------------------------------------------------------------------------------
link to paper -> https://arxiv.org/abs/1709.00155

-------------------------------------------------------------------------------------------------------------------
# Technology used: Tensorflow

as usual, I'll start with the utility cells:

In [1]:
# packages used for processing: 
import matplotlib.pyplot as plt # for visualization
import numpy as np

# for operating system related stuff
import os
import sys # for memory usage of objects
from subprocess import check_output

# The tensorflow_graph_package for this implementation
from Summary_Generator.Tensorflow_Graph.utils import *
from Summary_Generator.Text_Preprocessing_Helpers.pickling_tools import *

# import tensorflow temporarily:
import tensorflow as tf

# to plot the images inline
%matplotlib inline

In [2]:
# Input data files are available in the "../Data/" directory.

def exec_command(cmd):
    '''
        function to execute a shell command and see it's 
        output in the python console
        @params
        cmd = the command to be executed along with the arguments
              ex: ['ls', '../input']
    '''
    print(check_output(cmd).decode("utf8"))

In [3]:
# check the structure of the project directory
exec_command(['ls', '..'])

Data
LICENSE
Literature
README.md
Scripts
TensorFlow_implementation



In [4]:
np.random.seed(3) # set this seed for a device independant consistent behaviour

In [5]:
''' Set the constants for the script '''

# various paths of the files
data_path = "../Data" # the data path

data_files_paths = {
    "table_content": os.path.join(data_path, "train.box"),
    "nb_sentences" : os.path.join(data_path, "train.nb"),
    "train_sentences": os.path.join(data_path, "train.sent")
}

base_model_path = "Models"
plug_and_play_data_file = os.path.join(data_path, "plug_and_play.pickle")

# constants for this script
train_percentage = 90

## Unpickle the processed data file and create the train_dev pratitions for it

In [6]:
data = unPickleIt(plug_and_play_data_file)

In [7]:
field_encodings = data['field_encodings']
field_dict = data['field_dict']

content_encodings = data['content_encodings']

label_encodings = data['label_encodings']
content_label_dict = data['content_union_label_dict']

## create a randomized cell that prints a complete sample to verify the sanity of the processed data

In [13]:
total_samples = len(field_encodings)

random_index = np.random.randint(total_samples)

# extract the three parts of this random sample
random_field_sample = field_encodings[random_index]
content_sample = content_encodings[random_index]
label_sample = label_encodings[random_index]

# print the extracted sample in meaningful format
print("Table Contents: ")
print([(field_dict[field], content_label_dict[content]) 
       for (field, content) in zip(random_field_sample, content_sample)])

print("\n")
print("Summary: ")
print([content_label_dict[label] for label in label_sample])

Table Contents: 
[('type', 'pope'), ('name', 'michael'), ('name', 'iii'), ('name', 'of'), ('name', 'alexandria'), ('title', '56th'), ('title', 'pope'), ('title', 'of'), ('title', 'alexandria'), ('title', '&'), ('title', 'patriarch'), ('title', 'of'), ('title', 'the'), ('title', 'see'), ('title', 'of'), ('title', 'st.'), ('title', 'mark'), ('image', '<none>'), ('caption', '<none>'), ('enthroned', '25'), ('enthroned', 'april'), ('enthroned', '880'), ('ended', '16'), ('ended', 'march'), ('ended', '907'), ('predecessor', 'shenouda'), ('predecessor', 'i'), ('successor', 'gabriel'), ('successor', 'i'), ('ordination', '<none>'), ('consecration', '<none>'), ('birthdate', '<none>'), ('birthname', '<none>'), ('birthplace', 'egypt'), ('deathdate', '16'), ('deathdate', 'march'), ('deathdate', '907'), ('buried', 'monastery'), ('buried', 'of'), ('buried', 'saint'), ('buried', 'macarius'), ('buried', 'the'), ('buried', 'great'), ('nationality', 'egyptian'), ('religion', 'coptic'), ('religion', 'ortho

run the above cell multiple times to satisfy yourself that the data is still sane.

## Perform random shuffling of the input data

In [14]:
X, Y = synch_random_shuffle_non_np(zip(field_encodings, content_encodings), label_encodings)

## Perform train_dev_splitting of the given data:

In [15]:
train_X, train_Y, dev_X, dev_Y = split_train_dev(X, Y, train_percentage)

In [16]:
print("Number of Examples in Training set: ", len(train_X))
print("Number of Examples in the dev  set: ", len(dev_X))

('Number of Examples in Training set: ', 9)
('Number of Examples in the dev  set: ', 1)


# Building graph temporarily:

step 0: Set the Hyper constants for the graph building process

In [31]:
# Set some hyper constants to be used in the graph building:

# random_seed value for consistent debuggable behaviour
seed_value = 3

# vocabulary sizes
field_vocab_size = data['field_vocab_size']
content_label_vocab_size = data['content_label_vocab_size']

# Embeddings size:
field_embedding_size = 100
content_label_embedding_size = 400 # This is a much bigger vocabulary compared to the field_name's vocabulary

# LSTM hidden state sizes
lstm_cell_state_size = hidden_state_size = 500 # they are same (for now)

In [533]:
# graph reset point:
tf.reset_default_graph()

step 1: Create placeholders for the computations in the graph

In [534]:
# Placeholders for the input data:
with tf.variable_scope("Input_Data"):
    tf_field_encodings = tf.placeholder(tf.int32, shape=(None, None), name="input_field_encodings")
    tf_content_encodings = tf.placeholder(tf.int32, shape=(None, None), name="input_content_encodings")
    tf_label_encodings = tf.placeholder(tf.int32, shape=(None, None), name="input_label_encodings")
    
    # This is a placeholder for storing the lengths of the input sequences (they are padded to tensor)
    tf_input_seqs_lengths = tf.placeholder(tf.int32, shape=(None,), name="input_sequence_lengths")
    
    # This is a placeholder for storing the lengths of the decoder sequences (they are padded to tensor)
    tf_label_seqs_lengths = tf.placeholder(tf.int32, shape=(None,), name="decoder_sequence_lengths")

In [535]:
# check tf_field_encodings
print(tf_field_encodings)

Tensor("Input_Data/input_field_encodings:0", shape=(?, ?), dtype=int32)


step 2: Obtain Embeddings for the input and the output sequences

In [536]:
# Scope for the shared Content_Label matrix
with tf.variable_scope("Unified_Vocabulary_Matrix"):
    content_label_embedding_matrix = tf.get_variable("content_label_embedding_matrix", 
                                shape=(content_label_vocab_size, content_label_embedding_size), 
                                initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value),
                                dtype=tf.float32)

In [537]:
# Embeddings for the given input data:
with tf.variable_scope("Input_Embedder"):
    # Embed the field encodings:
    field_embedding_matrix = tf.get_variable("field_embedding_matrix", 
                                shape=(field_vocab_size, field_embedding_size), 
                                initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value),
                                dtype=tf.float32)
    
    tf_field_embedded = tf.nn.embedding_lookup(field_embedding_matrix, tf_field_encodings, name="field_embedder")
    
    # Embed the content encodings: 
    
    
    tf_content_embedded = tf.nn.embedding_lookup(content_label_embedding_matrix, 
                                                 tf_content_encodings, name="content_embedder")

In [538]:
print("Embedded_Input_Tensors: ", tf_field_embedded, tf_content_embedded)

('Embedded_Input_Tensors: ', <tf.Tensor 'Input_Embedder/field_embedder:0' shape=(?, ?, 100) dtype=float32>, <tf.Tensor 'Input_Embedder/content_embedder:0' shape=(?, ?, 400) dtype=float32>)


In [539]:
# Embeddings for the label (summary sentences):
with tf.variable_scope("Label_Embedder"):
    # embed the label encodings
    tf_label_embedded = tf.nn.embedding_lookup(content_label_embedding_matrix, 
                                                 tf_label_encodings, name="label_embedder")

In [540]:
print("Embedded_Label_Tensors: ", tf_label_embedded)

('Embedded_Label_Tensors: ', <tf.Tensor 'Label_Embedder/label_embedder:0' shape=(?, ?, 400) dtype=float32>)


In [541]:
# Concatenate the Input embeddings channel_wise and obtain the combined input tensor
with tf.variable_scope("Input_Concatenator"):
    tf_field_content_embedded = tf.concat([tf_field_embedded, tf_content_embedded], axis=-1, name="concatenator")

In [542]:
print("Final_Input_to_the_Encoder: ", tf_field_content_embedded)

('Final_Input_to_the_Encoder: ', <tf.Tensor 'Input_Concatenator/concatenator:0' shape=(?, ?, 500) dtype=float32>)


step 3: Create the encoder RNN to obtain the encoded input sequences. <b>(The Encoder Module)</b>

In [543]:
with tf.variable_scope("Encoder"):
    encoded_input, encoder_final_state = tf.nn.dynamic_rnn (
                            cell = tf.nn.rnn_cell.LSTMCell(lstm_cell_state_size), # let all parameters to be default
                            inputs = tf_field_content_embedded,
                            sequence_length = tf_input_seqs_lengths,
                            dtype = tf.float32
                        )

In [544]:
print("Encoded_vectors_bank for attention mechanism: ", encoded_input)

('Encoded_vectors_bank for attention mechanism: ', <tf.Tensor 'Encoder/rnn/transpose:0' shape=(?, ?, 500) dtype=float32>)


In [545]:
# define the size parameter for the encoded_inputs
encoded_inputs_embeddings_size = encoded_input.shape[-1]
print encoded_inputs_embeddings_size

500


In [546]:
print("Final_state obtained from the last step of encoder: ", encoder_final_state)

('Final_state obtained from the last step of encoder: ', LSTMStateTuple(c=<tf.Tensor 'Encoder/rnn/while/Exit_2:0' shape=(?, 500) dtype=float32>, h=<tf.Tensor 'Encoder/rnn/while/Exit_3:0' shape=(?, 500) dtype=float32>))


step 4: define the Attention Mechanism for the Model <b>(The Dispatcher Module)</b>

step 4.1: define the content based attention

In [547]:
with tf.variable_scope("Content_Based_Attention"):
    '''
        These weights and bias matrices must be compatible with the dimensions of the h_values and the f_values
        passed to the function below. If they are not, some exception might get thrown and it would be difficult
        to debug it. 
    '''
    # field weights for the content_based attention
    W_f = tf.get_variable("field_attention_weights", shape=(field_embedding_size, content_label_embedding_size),
                         initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value))
    b_f = tf.get_variable("field_attention_biases", shape=(field_embedding_size,),
                         initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value))
    
    # hidden states weights for the content_based attention
    W_c = tf.get_variable("content_attention_weights", 
                          shape=(encoded_inputs_embeddings_size, content_label_embedding_size),
                          initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value))
    b_c = tf.get_variable("content_attention_biases", shape=(encoded_inputs_embeddings_size,),
                          initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value))

In [548]:
with tf.variable_scope("Content_Based_Attention"):
    def get_content_based_attention_vectors(query_vectors):
        '''
            function that returns the alpha_content vector using the yt-1 (query vectors)
        '''
        # use the W_f and b_f to transform the query_vectors to the shape of f_values
        f_trans_query_vectors = tf.matmul(W_f, tf.transpose(query_vectors)) + b_f
        # use the W_c and b_c to transform the query_vectors to the shape of h_values
        h_trans_query_vectors = tf.matmul(W_c, tf.transpose(query_vectors) + b_c)
        
        # use the tf.while_loop to obtain the required vectors
        field_attention_values = tf.TensorArray(tf.float32, size=f_trans_query_vectors.shape[-1])
        # use the tf.while_loop to obtain the required vectors
        hidden_attention_values = tf.TensorArray(tf.float32, size=h_trans_query_vectors.shape[-1])
        tf.while_loop (
            lambda i, x, y: tf.less(i, field_attention_values.size()),
            lambda i, x, y: (tf.add(i, 1), x.write(i, tf.matmul(tf_field_embedded[i, :, :],
                                                        tf.transpose(tf.stack([f_trans_query_vectors[:, i]])))),
                               y.write(i, tf.matmul(encoded_input[i, :, :], 
                                                        tf.transpose(tf.stack([h_trans_query_vectors[:, i]]))))),
            [tf.constant(0), field_attention_values, hidden_attention_values],
            name="attention_loop"
        )
        
        field_attention_values = tf.squeeze(field_attention_values.stack())
        hidden_attention_values = tf.squeeze(hidden_attention_values.stack())
        
        # return the element wise multiplied values followed by softmax
        return tf.nn.softmax(field_attention_values * hidden_attention_values, name="softmax")

step 4.2: define the link based attention

In [549]:
with tf.variable_scope("Link_Based_Attention"):
    '''
        The dimensions of the Link_Matrix must be properly compatible with the field_vocab_size.
        If they are not, some exception might get thrown and it would be difficult
        to debug it.
    '''
    Link_Matrix = tf.get_variable("Link_Attention_Matrix", shape=(field_vocab_size, field_vocab_size),
            dtype=tf.float32, initializer=tf.truncated_normal_initializer(mean=0.5, stddev=0.5, seed=seed_value))

In [550]:
print(Link_Matrix)

<tf.Variable 'Link_Based_Attention/Link_Attention_Matrix:0' shape=(106, 106) dtype=float32_ref>


In [551]:
# define the function for obtaining the link based attention values.
with tf.variable_scope("Link_Based_Attention"):
    def get_link_based_attention_vectors(prev_attention_vectors):
        '''
            This function generates the link based attention vectors using the Link matrix and the 
        '''
        # carve out only the relevant values from the Link matrix
        
        
        matrix_all_values_from = tf.nn.embedding_lookup(Link_Matrix, tf_field_encodings)
        matrix_relevant_values = tf.transpose(
                        tf.gather_nd(tf.transpose(matrix_all_values_from, perm=[0, 2, 1]), tf_field_encodings), 
                        perm=[0, 2, 1])
        
        return tf.nn.softmax(tf.reduce_sum(prev_attention_vectors * matrix_relevant_values, axis=1),name="softmax")

step 4.3: define the hybrid attention

In [552]:
# define the hybrid of the content based and the link based attention
with tf.variable_scope("Hybrid_attention"):
    # for now, this is just the content_based attention:
    Zt_weights = tf.get_variable("zt_gate_parameter_vector", dtype=tf.float32,
                                 initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value),
                                 shape=(hidden_state_size + field_embedding_size + content_label_embedding_size, 1))

In [553]:
with tf.variable_scope("Hybrid_attention"):
    # define the hybrid_attention_calculator function:
    def get_hybrid_attention(h_values, y_values, content_attention, link_attention):
        '''
            function to calcuate the hybrid attention using the content_attention and the link_attention
        '''
        # calculate the e_f values
        e_t = tf.reduce_sum(link_attention * tf_field_embedded, axis=1)
        
        # create the concatenated vectors from h_values e_t and y_values
        input_to_zt_gate = tf.concat([h_values, e_t, y_values], axis=-1) # channel wise concatenation
        
        # perfrom the computations of the z gate:
        z_t = tf.nn.sigmoid(tf.matmul(input_to_zt_gate, Zt_weights))
        
        # calculate z_t~ value using the empirical values = 0.2z_t + 0.5
        z_t_tilde = (0.2 * z_t) + 0.5
        
        # compute the final hybrid_attention_values using the z_t_tilde values over content and link based values
        hybrid_attention = (z_t_tilde * content_attention) + ((1 - z_t_tilde) * link_attention)
        
        # return the calculated hybrid attention:
        return hybrid_attention

step 5: create the decoder RNN to obtain the generated summary for the structured data <b>(The Decoder Module)</b>

In [554]:
 with tf.variable_scope("Decoder"):
        # define the weights for the output projection calculation
        W_output = tf.get_variable(
                            "output_projector_matrix", dtype=tf.float32,
                            initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value),
                            shape=(hidden_state_size, content_label_vocab_size))
        b_output = tf.get_variable(
                            "output_projector_biases", dtype=tf.float32,
                            initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value),
                            shape=(content_label_vocab_size,))
        
        # define the weights and biases for the x_t calculation
        W_d = tf.get_variable(
                        "x_t_gate_matrix", dtype=tf.float32,
                        initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value),
                        shape=((hidden_state_size + content_label_embedding_size), content_label_embedding_size))
        b_d = tf.get_variable(
                            "x_t_gate_biases", dtype=tf.float32,
                            initializer=tf.random_uniform_initializer(minval=-1, maxval=1, seed=seed_value),
                            shape=(content_label_embedding_size,))

In [555]:
def decode(start_tokens, mode = "inference", decoder_lengths = None, w_reuse = True):
    '''
        Function that defines the decoder op and returns the decoded sequence (the summary)
        
        @params:
        start_tokens = a tensor containing the start tokens (one for each sequence in the batch)
        mode = a value from "training" or "inference" to determine for how long the decoder rnn is to be unrolled.
               behaviour is as follows:
               "training" => The rnn will be unrolled until the max(decode_lengths). decode_lengths cannot be None.
               "inference" => decode_lengths is be ignored and unrolling will be done till <eos> is received
               
    '''
    with tf.variable_scope("Decoder", reuse = w_reuse):
        # define the function to obtain the predictions out of the given hidden_state_values
        def get_predictions(h_t_values):
            '''
                This function transforms the h_t_values into a one_hot_type probability vector
            '''
            # apply the output_projection gate to obtain the predictions from the h_t_values
            predictions = tf.matmul(h_t_values, W_output) + b_output
            
            # return the predictions:
            return predictions
        
        
        # define a function to obtain the values for the next input to the LSTM_cell (y_t values)
        def get_y_t_values(pred_vals):
            '''
                pred_vals = the tensor of shape [batch_size x content_label_vocab_size]
            '''
            
            # calculate the next words to be predicted 
            act_preds = tf.argmax(pred_vals, axis=-1)
            
            # perform embedding lookup for these act_preds
            y_t_values = tf.nn.embedding_lookup(content_label_embedding_matrix, act_preds)
            
            # return the calculated y_t_values
            return y_t_values
            
        
        # write the loop function for the raw_rnn:
        def decoder_loop_function(time, cell_output, cell_state, loop_state):
            '''
                The decoder loop function for the raw_rnn
                (In future will implement the attention mechanism using the loop_state parameter.)
                @params
                compatible with -> https://www.tensorflow.org/api_docs/python/tf/nn/raw_rnn
            '''
            if(cell_state is None):
                # initial call of the loop function
                finished = (time >= tf_label_seqs_lengths)
                next_input = start_tokens
                next_cell_state = encoder_final_state
                emit_output = None
                next_loop_state = tf.zeros_like(tf_field_encodings, dtype=tf.float32)

            else:
                # we define the loop_state as the prev_hybrid attention_vector!
                prev_attention_vectors = loop_state # extract the prev_attention_vector from the loop state
                
                # obtain the predictions for the cell_output
                preds = get_predictions(cell_output)
                
                # obtain the y_t_values from the cell_output values:
                y_t_values = get_y_t_values(preds)
                
                ''' Calculate the attention: '''
                # calculate the content_based attention values using the defined module
                cont_attn = get_content_based_attention_vectors(y_t_values)
                
                # calculate the link based attention values
                link_attn = get_link_based_attention_vectors(prev_attention_vectors)
                
                # calculate the hybrid_attention
                hybrid_attn = get_hybrid_attention(cell_output, y_t_values, cont_attn, link_attn)
                
                ''' Calculate the x_t vector for next_input value'''
                # use the hybrid_attn to attend over the encoded_input (to calculate the a_t values)
                a_t_values = tf.reduce_sum(hybrid_attn * encoded_input, axis=1) 
                
                # apply the x_t gate
                x_t = tf.tanh(tf.matmul(tf.concat([a_t_values, y_t_values], axis=-1), W_d) + b_d)
                
                
                ''' Calculate the finished vector for perfoming computations '''
                # for now it is just the decoder length completed or not value.
                finished = (time >= decoder_lengths)
                
                ''' Copy mechanism is left (//TODO: change the following and implement copy mechanism)'''
                emit_output = preds
                
                # The next_input is the x_t vector so calculated:
                next_input = x_t
                # The next loop_state is the current hybrid_attention vectors
                next_loop_state = hybrid_attn
                # The next_cell_state is going to be equal to the cell_state. (we_don't tweak it)
                next_cell_state = cell_state
            
            # In both the cases, the return value is same.
            # return all these created parameters
            return (finished, next_input, next_cell_state, emit_output, next_loop_state)
        
        # use the tf.nn.raw_rnn to define the decoder computations
        outputs, _, _ = tf.nn.raw_rnn(tf.nn.rnn_cell.LSTMCell(lstm_cell_state_size), decoder_loop_function)
        
    # return the outputs obtained from the raw_rnn:
    return outputs

step 6: define the training computations:

In [556]:
with tf.variable_scope("Training_computations"):
    outputs = decode(tf_label_embedded[:, 0, :], mode="training", 
                     decoder_lengths=tf_label_seqs_lengths, w_reuse=None)

ValueError: Dimension 1 in both shapes must be equal, but are 500 and 421 for 'Training_computations/Decoder/rnn/while/Select' (op: 'Select') with input shapes: [?], [?,500], [?,421].

step _ : define the errands for the model

In [557]:
with tf.variable_scope("Errands"): 
    init = tf.global_variables_initializer()
    all_summaries = tf.summary.merge_all()

## Create a stub_session to generate the graph visualization

In [558]:
model_name = "Model_1"

In [559]:
model_path = os.path.join(base_model_path, model_name)

In [560]:
with tf.Session() as sess:
    tensorboard_writer = tf.summary.FileWriter(model_path, graph=sess.graph, filename_suffix=".bot")
    
    # initialize the session to generate the visualization file
    sess.run(init)
    
    tvars = tf.trainable_variables()
    tvars_vals = sess.run(tvars)
    
    for var, val in zip(tvars, tvars_vals):
        print(var.name)

Unified_Vocabulary_Matrix/content_label_embedding_matrix:0
Input_Embedder/field_embedding_matrix:0
Encoder/rnn/lstm_cell/kernel:0
Encoder/rnn/lstm_cell/bias:0
Content_Based_Attention/field_attention_weights:0
Content_Based_Attention/field_attention_biases:0
Content_Based_Attention/content_attention_weights:0
Content_Based_Attention/content_attention_biases:0
Link_Based_Attention/Link_Attention_Matrix:0
Hybrid_attention/zt_gate_parameter_vector:0
Decoder/output_projector_matrix:0
Decoder/output_projector_biases:0
Decoder/x_t_gate_matrix:0
Decoder/x_t_gate_biases:0
Training_computations/Decoder/rnn/lstm_cell/kernel:0
Training_computations/Decoder/rnn/lstm_cell/bias:0
